In [21]:
import dask.dataframe as dd
from dask import delayed
from fastparquet import ParquetFile
import dask_ml.cluster as dmlc
from dask_ml.preprocessing import MinMaxScaler
import glob

@delayed
def load_chunk(pth):
    return ParquetFile(pth).to_pandas()

In [2]:
wd = '/scratch/midway3/graziul/Zone1/'

In [3]:
data_files_list = glob.glob(wd+'*/*gemapsv01b.parquet')
len(data_files_list)

360

In [4]:
data_files_list[0]

'/scratch/midway3/graziul/Zone1/2019_07_09/2019_07_09gemapsv01b.parquet'

In [5]:
from dask.distributed import Client, progress
client = Client(n_workers=12, threads_per_worker=1, memory_limit='16GB')
client

Client Scheduler: tcp://127.0.0.1:40199 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 12 Cores: 12 Memory: 192.00 GB


In [6]:
feats = ['Loudness_sma3', 'alphaRatio_sma3',
       'hammarbergIndex_sma3', 'slope0-500_sma3', 'slope500-1500_sma3',
       'F0semitoneFrom27.5Hz_sma3nz', 'jitterLocal_sma3nz',
       'shimmerLocaldB_sma3nz', 'HNRdBACF_sma3nz',
       'logRelF0-H1-H2_sma3nz', 'logRelF0-H1-A3_sma3nz',
       'F1frequency_sma3nz', 'F1bandwidth_sma3nz',
       'F1amplitudeLogRelF0_sma3nz', 'F2frequency_sma3nz',
       'F2amplitudeLogRelF0_sma3nz', 'F3frequency_sma3nz',
       'F3amplitudeLogRelF0_sma3nz']

In [30]:
df = dd.from_delayed([load_chunk(f)[feats] for f in data_files_list[0:20]])

In [31]:
df = df.astype('float64')

In [32]:
scaler = MinMaxScaler()
dff = scaler.fit_transform(df)

In [33]:
dff.head()

,Loudness_sma3,alphaRatio_sma3,hammarbergIndex_sma3,slope0-500_sma3,slope500-1500_sma3,F0semitoneFrom27.5Hz_sma3nz,jitterLocal_sma3nz,shimmerLocaldB_sma3nz,HNRdBACF_sma3nz,logRelF0-H1-H2_sma3nz,logRelF0-H1-A3_sma3nz,F1frequency_sma3nz,F1bandwidth_sma3nz,F1amplitudeLogRelF0_sma3nz,F2frequency_sma3nz,F2amplitudeLogRelF0_sma3nz,F3frequency_sma3nz,F3amplitudeLogRelF0_sma3nz
index,,,,,,,,,,,,,,,,,,
0,0.891062,0.570824,0.636865,0.407161,0.711088,0.0,0.0,0.0,0.0,0.0,0.0,0.588832,0.169548,0.0,0.696418,0.0,0.620617,0.0
1,0.893882,0.561899,0.626436,0.476565,0.659140,0.0,0.0,0.0,0.0,0.0,0.0,0.599650,0.175540,0.0,0.715977,0.0,0.609349,0.0
2,0.881827,0.534235,0.642016,0.511808,0.576440,0.0,0.0,0.0,0.0,0.0,0.0,0.607914,0.170531,0.0,0.718447,0.0,0.605591,0.0
3,0.877848,0.517465,0.670805,0.535436,0.489329,0.0,0.0,0.0,0.0,0.0,0.0,0.621258,0.164732,0.0,0.743082,0.0,0.610873,0.0
4,0.877838,0.522198,0.695098,0.524874,0.457522,0.0,0.0,0.0,0.0,0.0,0.0,0.559799,0.148550,0.0,0.682990,0.0,0.589477,0.0


In [34]:
X = dff.to_dask_array(lengths=True)

In [35]:
X

dask.array<values, shape=(27799726, 18), dtype=float64, chunksize=(1917335, 18), chunktype=numpy.ndarray>

In [ ]:
dc = dmlc.SpectralClustering(n_clusters=10, n_components=100).fit(X)